In [1]:
%%html
<style>
  table { float: left; background: #d9d9d9; }
</style>

# gather_vm_factsメソッド

```python
def gather_vm_facts(content, vm)
```

vmを指定してvmのManaged Objectから仮想マシン情報を取得するメソッドです。  

## 引数

| args    | type                     | param                             |
| :-      | :----------------------- | :-------------------------------- |
| content | class                    | ServiceContent Object             |
| vm      | class                    | VMのManaged Object                |

## 戻り値

| rtype    | return    |
| :------- | :-------- |
| dict     | VMの情報  |


# gather_vm_factsメソッド動作確認

gather_vm_factsメソッドの動作確認をしてみます。  
以下の変数は環境に合わせて変更してください。

* vm_name
* hostname
* username
* password
* validate_certs

`key : value` で情報を出力します。

In [ ]:
from ansible.module_utils.vmware import connect_to_api, find_obj, gather_vm_facts
from pyVmomi import vim

vm_name = 'VM Name'

class module():
    def __init__(self):
        self.params = {}

m = module()
m.params['hostname'] = 'vCenter IP or HostName'
m.params['username'] = 'UserName'
m.params['password'] = 'Password'
m.params['validate_certs'] = False

sc = connect_to_api(m)
vm_obj = find_obj(sc, [vim.VirtualMachine], vm_name)
if(vm_obj):
    r = gather_vm_facts(sc, vm_obj)
    for key in r.keys():
        print("%s : %s" % (key, r[key]))
else:
    print("%s not found" % vm_name)

# gather_vm_factsメソッドを使ったモジュール例

gather_vm_factsを使って実際にAnsibleモジュールを作ってみましょう。  
以下のモジュールはVM名を指定してVMの情報を取得します。   
モジュールは `/tmp/ansible_salf_made_module` に `gather_vm_facts.py` で保存されます。  
`/tmp/ansible_salf_made_module` ディレクトリが無い場合は作成してください。  
既にディレクトリがある場合は、以下を実行してモジュールファイルを作成します。

In [ ]:
%%writefile /tmp/ansible_salf_made_module/gather_vm_facts.py
#!/usr/bin/python

try:
    from pyVmomi import vim, vmodl
    HAS_PYVMOMI = True
except ImportError:
    HAS_PYVMOMI = False

from ansible.module_utils.vmware import find_obj, gather_vm_facts, connect_to_api, vmware_argument_spec
from ansible.module_utils.basic import AnsibleModule

def main():
    argument_spec = vmware_argument_spec()
    argument_spec.update(name=dict(type='str', required=True))

    module = AnsibleModule(argument_spec=argument_spec, supports_check_mode=True)

    if not HAS_PYVMOMI:
        module.fail_json(msg='pyvmomi python library not found')

    vm_name = module.params['name']

    result = dict(changed=False)
    sc = connect_to_api(module)
    vm_obj = find_obj(sc, [vim.VirtualMachine], vm_name)
    if(vm_obj):
        r = gather_vm_facts(sc, vm_obj)
        result['vm_facts'] = r
        module.exit_json(**result)
    else:
        module.fail_json(msg="%s not found" % vm_name)

if __name__ == "__main__":
    main()

# モジュールを動かす

作ったモジュールを実際に動かしてみます。  
以下のオプションは環境に合わせて変更してください。

* hostname
* username
* password
* validate_certs
* name

変更後、Kernelを `Ansible` に変更して以下を実行します。

In [ ]:
#ansible.cfg
[defaults]
library = /tmp/ansible_salf_made_module

In [ ]:
#inventory
[all]
localhost

In [ ]:
#play
name: salf-made module test
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
gather_vm_facts:
  hostname: vCenter IP or HostName
  username: UserName
  password: Password
  validate_certs: no
  name: VM Name

`name` に指定したVMの情報が取得できていることを確認します。